# NB per la visualizzazione dell'output di WRF
Devo ancora vedere cosa bisogna mostrare come input e controllare bene quali sono gli output da visualizzare

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap 
from pathlib import Path
import PIL

In [2]:
# Only edit this cell

filepath = r"C:\Users\giova\Downloads\wrfout_d01_0001-01-01_00_00_00.nc"
savedirpath = r'C:\Users\giova\Desktop\TESTCMAP'
variables = [
    'FGRNHFX',
    'T2'
]

In [3]:
def get_array_from_fire_subgrid(var_name, root):
    sr_x=int(len(root.dimensions['west_east_subgrid'])/(len(root.dimensions['west_east'])+1))
    sr_y=int(len(root.dimensions['south_north_subgrid'])/(len(root.dimensions['south_north'])+1))
    data=root.variables[var_name]
    return np.array(data[:,:-sr_y,:-sr_x].data), sr_x, sr_y


def draw_cmap_at_time(data, lon, lat, time, title='CMAP'):
    path = savedirpath + '\\'+ title 
    Path(path).mkdir(parents=True, exist_ok=True)
    plt.figure(figsize=(10,10)) #figure size 
    x, y = np.meshgrid(lon,lat)
    c_scheme = plt.pcolor(x,y,np.squeeze(data[time,:,:]),cmap = 'jet', shading='auto') 
    cbar = plt.colorbar(c_scheme) # map information

    plt.title(title + ' ' + str(time))    
    plt.clim(data.min(),data.max())
    
    savepath = savedirpath + '\\'+ title + '\\' + str(time)+'.jpg'
    plt.savefig(savepath)
    plt.clf()
    plt.close()
    return savepath


def make_gif(data, lon, lat, title):
    print(f'Making GIF for {title}')
    image_frames = [] 
    dh = display('Frame 0',display_id=True)
    for i in range(data.shape[0]):
        dh.update(f'Frame {i+1}')
        path = draw_cmap_at_time(data, lon, lat, i, title)
        new_frame = PIL.Image.open(path) 
        image_frames.append(new_frame)

    image_frames[0].save(savedirpath + '\\'+ title + '_timelapse.gif',format='GIF',
                        append_images = image_frames[1: ],
                        save_all = True, duration = 100,
                        loop = 0)
    print('GIF saved at ' + savedirpath + '\\'+ title + '_timelapse.gif')
    

In [4]:
root = Dataset(filepath, "r", format="NETCDF4")

for var in variables:
    arr = np.array(root.variables[var][:].data)
    lats = root.variables['XLAT'][0, :, 0]
    longs = root.variables['XLONG'][0, 0, :]    
    if 'south_north_subgrid' in root.variables[var].dimensions:
        arr, sr_x, sr_y = get_array_from_fire_subgrid(var, root)
        longs = np.linspace(longs.min(), longs.max(), len(root.dimensions['west_east_subgrid'])-sr_x), 
        lats = np.linspace(lats.min(), lats.max(), len(root.dimensions['south_north_subgrid'])-sr_y)
    
    make_gif(arr, longs, lats, var)
    
print('ALL DONE!')

Making GIF for FGRNHFX


'Frame 61'

GIF saved at C:\Users\giova\Desktop\TESTCMAP\FGRNHFX_timelapse.gif
Making GIF for T2


'Frame 61'

GIF saved at C:\Users\giova\Desktop\TESTCMAP\T2_timelapse.gif
ALL DONE!
